In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [5]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_18752\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [6]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_18752\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [7]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
le = LabelEncoder()

In [10]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [11]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [12]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [13]:
df.cat.tail(20)

,Soil Type,Crop Type,Fertilizer Name
749980,0,10,5
749981,2,2,2
749982,3,8,0
749983,4,3,2
749984,4,6,1
749985,3,3,0
749986,0,10,3
749987,3,3,6
749988,0,3,1
749989,0,4,4


In [14]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [15]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]].values

In [16]:
y = df_train[["Fertilizer Name"]].values

In [17]:
import torch

x_tensor = torch.tensor(X, dtype=torch.float)  # X is your feature DataFrame
y_tensor = torch.tensor(y, dtype=torch.long).squeeze()  # y is your label DataFrame

In [18]:
y = y_tensor.cpu().numpy().ravel()

In [19]:
X = x_tensor.cpu().numpy()

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
RFC=RandomForestClassifier(n_estimators=8)

In [23]:
fit=RFC.fit(X,y)

In [24]:
RFC.fit(X,y)

RandomForestClassifier(n_estimators=8)

In [25]:
fit.score(X,y)

0.6726066666666667

In [27]:

RFC_preds = RFC.predict(X)
X_np = x_tensor.cpu().numpy()
y_np = y.values if hasattr(y, 'values') else y  # if y is a pandas Series
y_tensor = torch.tensor(y_np, dtype=torch.long).to(device)

In [28]:
import torch.nn as nn
class SimpleGCN(nn.Module):
    def __init__(self, in_channels, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.conv3(x, edge_index)

In [29]:
from torch_geometric.nn import GCNConv

In [30]:
import torch.nn as nn
class SimpleGCN(nn.Module):
    def __init__(self, in_channels, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.conv3(x, edge_index)

In [31]:
model = SimpleGCN(in_channels=x_tensor.shape[1], hidden_dim=128, num_classes=len(torch.unique(y_tensor))).to(device)

In [32]:
from sklearn.neighbors import NearestNeighbors

def build_class_filtered_edge_index(X_np, rf_preds, n_neighbors=5, max_edges_per_class=None):
    """
    Builds an edge_index tensor using k-NN within RF-predicted class groups.
    
    Args:
        X_np (np.ndarray): Feature matrix (NumPy) of shape (n_samples, n_features)
        rf_preds (np.ndarray): RF predicted class labels for each sample (n_samples,)
        n_neighbors (int): Number of neighbors to connect per node (within class)
        max_edges_per_class (int or None): Cap per class edge count for memory safety

    Returns:
        edge_index (torch.LongTensor): 2 x E tensor of edges
    """
    all_edges = []

    for cls in np.unique(rf_preds):
        class_idx = np.where(rf_preds == cls)[0]
        if len(class_idx) <= 1:
            continue  # not enough points to connect

        class_features = X_np[class_idx]
        try:
            nbrs = NearestNeighbors(n_neighbors=min(n_neighbors + 1, len(class_idx)), metric="cosine").fit(class_features)
            _, indices = nbrs.kneighbors(class_features)
        except Exception as e:
            print(f"[Warning] Skipped class {cls} due to error:", e)
            continue

        rows = np.repeat(class_idx, indices.shape[1] - 1)
        cols = class_idx[indices[:, 1:].flatten()]  # exclude self-loop at [0]
        edge_pairs = np.stack([rows, cols], axis=0)

        if max_edges_per_class:
            edge_pairs = edge_pairs[:, :max_edges_per_class]

        all_edges.append(edge_pairs)

    # Concatenate all classes' intra-class edges
    if not all_edges:
        raise ValueError("No valid class-wise edges constructed.")

    edge_index_np = np.concatenate(all_edges, axis=1)
    
    # Optional: Make undirected without duplication explosion
    edge_index_np = np.concatenate([edge_index_np, edge_index_np[[1, 0]]], axis=1)

    # Deduplicate edges (optional but recommended for stability)
    edge_index_np = np.unique(edge_index_np, axis=1)

    return torch.tensor(edge_index_np, dtype=torch.long)

In [33]:
edge_index = build_class_filtered_edge_index(
    X_np,
    RFC_preds,
    n_neighbors=5,              # Start light, increase if memory allows
    max_edges_per_class=1000  # Optional memory cap per class
).to(device)

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

In [38]:
import torch.nn.functional as F
model.train()
for epoch in range(1, 16):
    optimizer.zero_grad()
    out = model(x_tensor, edge_index)
    loss = F.cross_entropy(out, y_tensor)
    loss.backward()
    optimizer.step()

    acc = (out.argmax(dim=1) == y_tensor).float().mean().item()
    print(f"Epoch {epoch:02d} | Loss: {loss.item():.4f} | Accuracy: {acc:.4f}")

Epoch 01 | Loss: 4.7627 | Accuracy: 0.1501
Epoch 02 | Loss: 4.7633 | Accuracy: 0.1537
Epoch 03 | Loss: 4.4607 | Accuracy: 0.1521
Epoch 04 | Loss: 4.0071 | Accuracy: 0.1472
Epoch 05 | Loss: 3.5650 | Accuracy: 0.1392
Epoch 06 | Loss: 3.1090 | Accuracy: 0.1347
Epoch 07 | Loss: 2.8007 | Accuracy: 0.1370
Epoch 08 | Loss: 2.9841 | Accuracy: 0.1423
Epoch 09 | Loss: 3.2667 | Accuracy: 0.1455
Epoch 10 | Loss: 3.2390 | Accuracy: 0.1467
Epoch 11 | Loss: 2.9708 | Accuracy: 0.1468
Epoch 12 | Loss: 2.6702 | Accuracy: 0.1459
Epoch 13 | Loss: 2.4464 | Accuracy: 0.1440
Epoch 14 | Loss: 2.3153 | Accuracy: 0.1402
Epoch 15 | Loss: 2.2989 | Accuracy: 0.1432
